# Churn Fact Table

In this notebook, we will create a separate CSV file containing clean quality data related to the Churn Fact table.

Let's begin by loading the source dataset.

In [1]:
# dependencies

import pandas as pd

## Data Loading

Load source dataset.

In [2]:
df = pd.read_csv('../data/Customer-Churn-Records.csv')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


## Data Preparation

To prepare source data for ingestion, we will drop unrequired columns and check for data quality issues.

### Churn Fact Table Requirements

For the Churn Fact Table, we will require the following columns:

- `Customer Key`: Foreign key to the Customer Profile Dimension Table.
- `Banking Profile Key`: Foreign key to the Banking Profile Dimension Table.
- `Credit Profile Key`: Foreign key to the Credit Profile Dimension Table.
- `Location Key`: Foreign key to the Location Dimension Table.
- `Churn`: Binary column indicating whether the customer has churned or not.

### Drop Unrequired Columns

Drop unrequired columns from the source dataset.

In [3]:
# drop the columns that are not needed
df.drop([
  'RowNumber',
  'Surname',
  'CreditScore',
  'Gender',
  'Age',
  'HasCrCard',
  'EstimatedSalary',
  'Satisfaction Score',
  'Card Type', 
  'Point Earned',
  'Tenure', 
  'Balance', 
  'NumOfProducts', 
  'IsActiveMember',
  'Complain'
  ],
  axis=1,
  inplace=True)

# confirm columns
df.columns

Index(['CustomerId', 'Geography', 'Exited'], dtype='object')

### Add Required Columns

Add foreign keys to the source dataset.

- For customer key, we will simply rename the `Customer ID` column to `Customer Key`.
- For banking profile key, we will use the `bank_profile_key` column from the file `banking_profile_dim.csv`.
- For credit profile key, we will use the `credit_profile_key` column from the file `credit_profile_dim.csv`.
- For location key, we will use the `location_key` column from the file `location_dim.csv`.
- For churn, we will simply rename the `Exited` column to `Churn`.

In [4]:
# Customer Key and Churn
df.rename(columns={
  'CustomerId': 'customer_key',
  'Exited': 'churn'
  },
  inplace=True)

# Bank Profile Key
df_banking_profile = pd.read_csv('../data/banking_profile_dim.csv')
df['bank_profile_key'] = df_banking_profile['bank_profile_key']

# Credit Profile Key
df_credit_profile = pd.read_csv('../data/credit_profile_dim.csv')
df['credit_profile_key'] = df_credit_profile['credit_profile_key']

# Location Key
df_location = pd.read_csv('../data/location_dim.csv')
# Create a mapping dictionary from country to location_key
country_to_key = dict(zip(df_location['country'], df_location['location_key']))
# Map the country to location_key
df['location_key'] = df['Geography'].map(country_to_key)
# Drop the Geography column
df.drop(['Geography'], axis=1, inplace=True)

### Re-order columns

We will re-order the columns in the order expected for staging the data into the Churn Fact table.

In [5]:
# change the order of the columns
df = df[['customer_key', 'bank_profile_key', 'credit_profile_key', 'location_key', 'churn']]

### Data Quality

To ensure we don't have any missing values, we will check for null values in the required columns.

In [6]:
# confirm no missing values
df.isnull().sum()

customer_key          0
bank_profile_key      0
credit_profile_key    0
location_key          0
churn                 0
dtype: int64

We can see that we don't have any missing values.

Let's confirm the data types and the range of values, to ensure they are in the correct format, and to exclude any noisy data.

In [7]:
# confirm data types
df.dtypes

customer_key          int64
bank_profile_key      int64
credit_profile_key    int64
location_key          int64
churn                 int64
dtype: object

In [8]:
# check the range of values
df.describe()

,customer_key,bank_profile_key,credit_profile_key,location_key,churn
count,1.000000e+04,10000.00000,10000.00000,10000.000000,10000.000000
mean,1.569094e+07,5000.50000,5000.50000,1.749500,0.203800
std,7.193619e+04,2886.89568,2886.89568,0.830433,0.402842
min,1.556570e+07,1.00000,1.00000,1.000000,0.000000
25%,1.562853e+07,2500.75000,2500.75000,1.000000,0.000000
50%,1.569074e+07,5000.50000,5000.50000,1.000000,0.000000
75%,1.575323e+07,7500.25000,7500.25000,3.000000,0.000000
max,1.581569e+07,10000.00000,10000.00000,3.000000,1.000000


Now, that the quality of our data is verified, we have one last step to perform before we can save the dataset in a CSV file.

## Data Export

Finally, we will export the clean data to a new CSV file.

In [9]:
# save the prepared and verified data to separate file
df.to_csv('../data/churn_fact_table.csv', index=False)